# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cityData = pd.read_csv('output_data\cities.csv')
cityData_pd = pd.DataFrame(cityData)

#clear old (identical) index
cityData_pd = cityData_pd.drop(columns='Unnamed: 0')

cityData_pd.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,cape town,ZA,-33.9258,18.4232,57.99,94,0,3.44,1619211557
1,coruripe,BR,-10.1256,-36.1756,78.80,79,54,8.43,1619211557
2,adrar,MR,20.5022,-10.0711,83.97,8,0,7.74,1619211558
3,great yarmouth,GB,52.6083,1.7305,45.00,61,69,3.44,1619211558
4,hobart,AU,-42.8794,147.3294,51.80,71,40,10.36,1619211559


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

fig = gmaps.figure()

In [4]:
locations = cityData_pd[["Lat", "Lng"]].astype(float)

heat_layer = gmaps.heatmap_layer(locations, weights=cityData_pd['Humidity'], 
                                 dissipating=True, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [41]:
#my ideal weather conditions are:
#  - 68F <= Max Temp <= 82F
#  - Cloudiness <= 55%
#  - Humidity <= 45%   

hotel_df = cityData_pd.loc[(cityData_pd['Max Temp'] >= 68) & (cityData_pd['Max Temp'] <= 82) & 
                                       (cityData_pd['Cloudiness'] <= 55) & (cityData_pd['Humidity'] <= 45), :]
hotel_df.head(20)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
77,baghdad,IQ,33.3406,44.4009,78.80,27,0,6.91,1619211597
113,laguna,US,38.4210,-121.4238,75.99,45,1,11.50,1619211626
164,roebourne,AU,-20.7833,117.1333,75.33,26,0,11.50,1619211660
239,port hedland,AU,-20.3167,118.5667,69.80,37,0,8.05,1619211704
268,beni suef,EG,29.0731,31.0979,69.01,40,0,14.97,1619211719
298,santo angelo,BR,-28.2992,-54.2631,80.60,44,0,5.75,1619211735
318,nioro,GM,13.3500,-15.7500,81.14,41,3,12.77,1619211745
379,mount pleasant,US,32.7941,-79.8626,69.01,34,1,10.36,1619211778
392,general roca,AR,-39.0333,-67.5833,69.80,40,0,6.91,1619211785
416,saint george,US,37.1041,-113.5841,80.01,10,1,17.27,1619211558


In [42]:
hotel_df = hotel_df.reindex(columns = hotel_df.columns.tolist() 
                                  + ['Hotel Name'])
hotel_df['Hotel Name'] = ''
hotel_df.reset_index(drop=True, inplace=True)

hotel_df.head(20)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
0,baghdad,IQ,33.3406,44.4009,78.80,27,0,6.91,1619211597,
1,laguna,US,38.4210,-121.4238,75.99,45,1,11.50,1619211626,
2,roebourne,AU,-20.7833,117.1333,75.33,26,0,11.50,1619211660,
3,port hedland,AU,-20.3167,118.5667,69.80,37,0,8.05,1619211704,
4,beni suef,EG,29.0731,31.0979,69.01,40,0,14.97,1619211719,
5,santo angelo,BR,-28.2992,-54.2631,80.60,44,0,5.75,1619211735,
6,nioro,GM,13.3500,-15.7500,81.14,41,3,12.77,1619211745,
7,mount pleasant,US,32.7941,-79.8626,69.01,34,1,10.36,1619211778,
8,general roca,AR,-39.0333,-67.5833,69.80,40,0,6.91,1619211785,
9,saint george,US,37.1041,-113.5841,80.01,10,1,17.27,1619211558,


In [45]:
#Hit the Google Places API for each city's coordinates.
#Store the first Hotel result into the DataFrame.

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#locations = hotel_df[["Lat", "Lng"]].astype(float)

#try:
row_index = 0
hotel_names = []
for row in hotel_df.iterrows():
    
    location = str(hotel_df.at[row_index, 'Lat']) + ',' + str(hotel_df.at[row_index,'Lng'])       #df.at[0,'A']
    params = {
            "location": location,
            "radius": 5000,
            "type": "lodging",
            "key": g_key,
        }
    row_index += 1
    response = requests.get(base_url, params=params).json()
    print(json.dumps(response, indent=4))
    hotel_names.append()
#except:
#    print('Oops! Something went wrong')

{
    "html_attributions": [],
    "next_page_token": "ATtYBwIWWXOugxbzolR9eUMb-oLp68pcj41FsB2eQmPgpHX2EQWmaOE9FMWSf3lI0qgs3TPDVSzyBO3eT5k5yAnxj-EWwJHU8hv3vjSQ0RtVzu-xK7D-rqXwaPKK2QAosjqTh3Oehhx6rGRIKio_on1iVd2c0HeHUmv_E8WLRiDO_zhBe52XZfCeF70l5-v1UinSOQByi6emfCjxb_PuafDCHl6ujEJptBfhPlZYQ_7alg9jpK-3XNtGydTmvEIYuQpnSJHsF1-AxnOR1rB7nsjY1-CbYwnsr-9BRU4QYmHvdBKl6DI5NH1ioLOKGIX560_VHTsLivig6IjsS8NM5x3Y9BKDwbJsIzCUrktKy29geC5QvxBx2Ikrsgf-uNPbvwX7C8uVfFLs_Kn0q4E90HIbHs4T7MdjI73Ld5xrT65NcnX10h1qCT2MILHj",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 33.31866790000001,
                    "lng": 44.41505780000001
                },
                "viewport": {
                    "northeast": {
                        "lat": 33.3199318302915,
                        "lng": 44.4162666302915
                    },
                    "southwest": {
                        "lat": 33.31723

{
    "html_attributions": [],
    "next_page_token": "ATtYBwI5JASt2XYJQgW9txiiBYc0dhHDSG6gede0_MiB6jQ4MVgdJSB4MWIigA3z-zbsW2MTbuLXAdxFXDNBuHJMOoBAAQvRcIochZ8hizau0nawHzg2kAJLsKB4zliN5_D3x-zOqQjsQd9Fh1-vKXCkmHSX-IE9mAGrCLWpUoVU7CKfjTERgysNsBblJM16ZE9ntU7-rTv7hdarQgTqbeEp8vSGnLNN3dkSniv1O-zPyrzKdlYd95_b_5FGt1Yx05KEuzYfKw5NLwAmtNoq5jSjwdVKcl1ERCC8P-K0wUCouo2j5gPtugtEEKkg-aGNM-RfNwc7Q_fAcK4ZQeJ5s71hxqWLXmJbY3JvfKP-KHWINaf-v0nlTYDQ-UGdTegzAbsqDGnPOsYCgwL5WO5u-OoiXY844xJ1E9jrkWpR1-9Ogp2TxqRt9RPaMowh",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 38.4240827,
                    "lng": -121.3987067
                },
                "viewport": {
                    "northeast": {
                        "lat": 38.4255044802915,
                        "lng": -121.3974215197085
                    },
                    "southwest": {
                        "lat": 38.4228065197085,


{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -20.7767233,
                    "lng": 117.150608
                },
                "viewport": {
                    "northeast": {
                        "lat": -20.7753731197085,
                        "lng": 117.1518880302915
                    },
                    "southwest": {
                        "lat": -20.7780710802915,
                        "lng": 117.1491900697085
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Harding River Caravan Park",
            "photos": [
                {
                    "height": 281,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/103254850910259115580\">Har

{
    "html_attributions": [],
    "next_page_token": "ATtYBwJNIyGZDmKfNAw15Y0YR3i0MjLWupQ8PD1O-ISuRIQ_CfWhjo7jtonKxMCwaaMlKAJaeATabQM67E29OnQnM2byjI49Tpp9tGiPckx0-M3es7sHgLuLg_1O7mxOeAO1ESwxzPBkceCJBEroIZh5c4Y0jDFQcnrvL5LH1eJpvTmKetWni4rrx3ODJt28Yex5VUw_F54VLoRyIUhZZSILxQA_47NyOvZZNXeeurGy9tZD5V7HA_ce_0r0BLrtCkYgd-0CLaqt7vkur7Nk6lrcrhK1l26D8vnc31w6oCUaA42QMkHyXF_XXjcChLiInsmEVPuJRc3GzWKqacDctGHkXPpDHsuPqgjBpCWj1Wa0WThs3hcSrONHHRImPsWGJqrc_YI1wTNWLQnXd2sW6S6eRjP5k4RQJjjn4r4WwfdlCvvOm0s20tcCZ_19",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 29.0629692,
                    "lng": 31.1009248
                },
                "viewport": {
                    "northeast": {
                        "lat": 29.0642607302915,
                        "lng": 31.1020544302915
                    },
                    "southwest": {
                        "lat": 29.0615627697085,
    

{
    "html_attributions": [],
    "next_page_token": "ATtYBwJy3X676yCjm42s_Ki46AZQUvsLqSpfd6B0eLksm2DQmoJOLZGhHQrQLBZFDMGQOsA_MTxCSJdomCX9eT6SmZBQc5yxe15WnNgIeB8gwWjRZxPvHeDNnnlZVG6WIfeR_kELWbDlIxFwhzcSdv4M1g4SXqxM4ZbYWqqgtcbHLRvUrTCi-DUuioulw6MjIfm1V7-ypZaXAgruEDi64ZiugxLzTw83jQf0l_W4vJTZdaTavVft_hSfjeeqkEp3AG26rqmpZdyuPHuyGuObFMBmVOcZqUzLhLE3Wb7GZECbld2MHuhCjLr2Sc2CfgHwM7VdGNRQ2EreDI5aJqjQXBX-4xu_6mpeC0ao5V1OmoKPdDiXTaStvS7SuHt4KCAyc6ToECkk4gqYGFGpmsINaLnwVQmq16JMpWHygf8byqYH-uTr70xf2ROnQS-e",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -28.3055604,
                    "lng": -54.2744739
                },
                "viewport": {
                    "northeast": {
                        "lat": -28.30417086970849,
                        "lng": -54.2731553697085
                    },
                    "southwest": {
                        "lat": -28.3068688302915

...

{
    "html_attributions": [],
    "next_page_token": "ATtYBwKHpH_dOWq-ccG9h54j4HckpWi0OT5ocmbYLgrqz46tmOfPsl3BYnIzunsJuIkycaMjyhiLXGI98UEcyTLdw7sgPax3sv2nc2IzqI67nu0sJmFxsjsMR15OWJwaqQxSghNWqWl0m-dvOXKCYY2rzWZzMz-hR85FgneUWTurDDUjCwGuMrtnacawzg_Ji8XPDybszY6lbwV1R2OBAXYMyEIcc-NkhUb_XQPgeh3KYF6DM_AUAqu6E1akEORMwQEbr3DsSNLbfTiqVZzCjZl9uJK1oWj7FlLw1y1K-JAdUuidctRbf6adFoxuhZGlwbWBupr9pXPPx0hTxeq-12MvxWgd-JDwzU9Dn9pjDugCoC3LgLxPxt6HOasbPPAQteJXvFerNRRNt1xGzc_h8jqR-V-oZGN-M5rPqLZsAcqZxRxHSSDg-7keYL95",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 29.9464214,
                    "lng": 40.1931247
                },
                "viewport": {
                    "northeast": {
                        "lat": 29.9477378302915,
                        "lng": 40.1946261302915
                    },
                    "southwest": {
                        "lat": 29.9450398697085,
    

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -26.5018377,
                    "lng": 17.1619833
                },
                "viewport": {
                    "northeast": {
                        "lat": -26.5005896697085,
                        "lng": 17.1630036302915
                    },
                    "southwest": {
                        "lat": -26.5032876302915,
                        "lng": 17.1603056697085
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Bethanie Hotel",
            "opening_hours": {
                "open_now": false
            },
            "photos": [
                {
                    "height": 3024,
                    "html_attributions": [
                        "<a href

{
    "html_attributions": [],
    "next_page_token": "ATtYBwLZpMxfG3Ti4SsoZZQm_5kk5q2QmdmqSB23Q1NbMC-JQRjtj62YMcmMkylfvTaHh-hoClvUxAPodx90PylHqLraKgnVEIQZfiu6UfCV9imp8VnmazAeuBNtb5c0M4Vnx2Dlc2mHDKSatE6fsrTEggYJxVjYKkTW8AsG0TAhaYeZLgQTDRB4PlRj8iDCHWXxnsKqmrVjU6O12pRwYkhNd_Rg7Et_V7obGVg0j1u0fJrHaZp54xwijsY4_Gl6RkKbQO8MgVU2TgdDX_94E9-eeuPMvyhGOTWVBMRadGKoA1DUglGWf2nNF2i-G2Y-utS_8Z0CL4yHcSo2VQZ4aBlgut_g7neklqsEdOvMLW4fKtCvGKdq0vSq93x_IHlZzkWU2G1sr4V8YgTeOvyptxpiuNtH4fm4lFLhsPIMGZwlHsXKb_0EjS_HlxPI",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 37.6054988,
                    "lng": -120.9717231
                },
                "viewport": {
                    "northeast": {
                        "lat": 37.6065432802915,
                        "lng": -120.9703850697085
                    },
                    "southwest": {
                        "lat": 37.6038453197085,


{
    "html_attributions": [],
    "next_page_token": "ATtYBwJBXuLgOg1lNiGk1QPKndE7LHPEzkHXfFGFKLb7cqAUR5ymHD0akd_Go9IMXQmAVg81FRMG8f8Pp7Hwi2CqMcv1kaH7I-kpijhnzgGng4M_b7pgSm0cWYU65hPfr1DkkeHPiK6HENQ0ABjx6XOQOi9lWuY2obujIay4tr2QhMGnmUHM5N2bYgnsRQUyfpjYtgvl3mPoMdtLiE57YjCnH1L9zUM1hMOWdclZ7omgPaQKyhyCXu6fsnNOZxKzvw6imElk6GFDRDHwuqBg_KQoU-sIh9x9xGh-Opwc0X8EEF4H5ZYsiTBTy_5ftckecBDbxeJratuwNHji_gzdUah3rD3Sbm_VSbXU-P2dcIVA34WYNd-s0wJPpqKXIpBaYryEQ3-HwSVOBpHfOAYAf5ZacS4pv4VaQS0utsnQ4ATGjyyqdN9bA5Sfz_Dg",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -21.6832417,
                    "lng": -49.76020339999999
                },
                "viewport": {
                    "northeast": {
                        "lat": -21.6818345697085,
                        "lng": -49.75892966970849
                    },
                    "southwest": {
                        "lat": -21.684532

{
    "html_attributions": [],
    "next_page_token": "ATtYBwLbLTlOpOdw5IGCuhncgeOklbWXkGwRw0sQcN8XEMASESwXjGLOayu_Qmz1FiEsDJVedDJPpIAENpFZMou1GTknr3bKJoFbKPeHEAhnktV5J2lb4wBl5TmEw-ZT9BFupRKkTlNrSNIwynLH01_dTIYxlbykhWIhA_5i8HQazNNmu2nfwO8SnIOvgSkIMItuaYLVWEG5_HpxNDtV2Ac2ixPuoEb8Ho7lgsNyul4jVKlV2uttVw5mC3iawPWxFfyTSWXYYsTjU8PckvAC1ww-4jYvB58mo_XNfmE0Us4MYBP-hExzegzt2X9hIaxu3ORgpduEITFWf_BXIB-P71_NRgGrnuhvAtuxjUAR1BITT0dLlHjnKkgIi94ZvctIswTA1py_RpW0XneDI2Dk8PMsn68oaEsY2KATulBcP_9SkL1A2XpwyvfOLGiQ",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 25.4478613,
                    "lng": 74.5730811
                },
                "viewport": {
                    "northeast": {
                        "lat": 25.4492115802915,
                        "lng": 74.5744214802915
                    },
                    "southwest": {
                        "lat": 25.44651361970849,
   

In [8]:
#NOTE: Do not change any of the code in this cell

#using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

#store the DataFrame Row
#NOTE: be sure to update with your DataFrame name

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
#NOTE: Do not change any of the code in this cell

#using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

#store the DataFrame Row
#NOTE: be sure to update with your DataFrame name

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
marker_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(22, 226, 109, 0.67)',
    stroke_color='rgba(22, 226, 109, 0.9)', scale=2,
    info_box_content=[hotel_info]
)

fig.add_layer(marker_layer)


# Display figure
fig

IndexError: list index out of range